In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats

import sys
sys.path.append('../src')
from data_imports import *

In [ ]:
def import_cbtn_TP53():
    path="data/pbta_TP53_alterations.xlsx"
    df = pd.read_excel(path)
    df = df.replace("not profiled",np.nan)
    df = df.dropna(subset=df.columns[4:], how='all')
    # aggregate to patient level
    gb = df.groupby('Patient ID')['Altered'].sum().reset_index()
    gb['TP53_altered'] = gb.Altered > 0
    gb = gb.set_index("Patient ID")
    return gb['TP53_altered']

def import_cbtn_TP53_pathway():
    path="data/alterations_across_samples.tsv"
    df = pd.read_csv(path,sep='\t')
    df = df.replace("not profiled",np.nan)
    df = df.dropna(subset=df.columns[4:], how='all')
    gb = df.groupby('Patient ID')['Altered'].sum().reset_index()
    gb['TP53_altered'] = gb.Altered > 0
    gb = gb.set_index("Patient ID")
    return gb['TP53_altered']

def import_tp53_pt():
    pt = import_patients()
    pt = pt.merge(import_cbtn_TP53_pathway(),how='inner',left_index=True,right_index=True)
    return pt



In [ ]:
pt = import_tp53_pt()
p53_mut_tumors = ['CHDM','EMBT','ETMR','HGG','MBL','MST','SARC']
#p53_mut_tumors=['HGG']
pt=pt[pt.cancer_type.isin(p53_mut_tumors)]
contingency_tbl = pd.crosstab(pt.amplicon_class,pt.TP53_altered)
print(
    'ecDNA vs intrachromosomal:',
    scipy.stats.chi2_contingency(contingency_tbl.loc[['ecDNA','intrachromosomal']])
)
print(
    'ecDNA vs no amplification',
    scipy.stats.chi2_contingency(contingency_tbl.loc[['ecDNA','no amplification']])
)

print(
    'intrachromosomal vs no amplification',
    scipy.stats.chi2_contingency(contingency_tbl.loc[['intrachromosomal','no amplification']])
)

print(contingency_tbl.sum(axis=1))

contingency_tbl

In [ ]:
pct_by_cancer_type = pd.crosstab(pt.cancer_type,pt.TP53_altered)
pct_by_cancer_type['frac'] = pct_by_cancer_type[True]/(pct_by_cancer_type[False]+pct_by_cancer_type[True])
pct_by_cancer_type
p53_mut_tumors = ['CHDM','EMBT','ETMR','HGG','MBL','MST','SARC']